In [55]:
# try to get 10 years of pollution data for Dehli from OpenAQ API

import openaq
import pandas as pd
from datetime import datetime, timedelta
import time

# Initialize OpenAQ API client
api = openaq.OpenAQ()

# Fetch data for Delhi
city = 'Delhi'
parameters = ['pm25', 'pm10', 'o3', 'co', 'so2', 'no2']

data_frames = []

end_date = datetime.now()
start_date = end_date - timedelta(days=365 * 10)  # 10 years ago

for parameter in parameters:
    current_start_date = start_date
    while current_start_date < end_date:
        current_end_date = current_start_date + timedelta(days=31)
        if current_end_date > end_date:
            current_end_date = end_date
        
        has_more_data = True
        page = 1
        while has_more_data:
            try:
                status, resp = api.measurements(city=city, parameter=parameter, limit=10000, page=page, date_from=current_start_date, date_to=current_end_date)
                if status == 200 and resp['results']:
                    df = pd.DataFrame(resp['results'])
                    data_frames.append(df)
                    if len(df) < 10000:
                        has_more_data = False
                    else:
                        page += 1
                else:
                    has_more_data = False
            except Exception as e:
                print(f"Error encountered: {e}. Retrying in 60 seconds.")
                time.sleep(60)

        current_start_date = current_end_date

# Merge data from all parameters
merged_data = pd.concat(data_frames, axis=0)



Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.


KeyboardInterrupt: 

In [56]:
# Create function to get data by year as was getting errors trying to request a full 10 years of data at once

def fetch_data_for_year(city, parameters, start_year):
    api = openaq.OpenAQ()
    year_data_frames = []
    
    for month in range(1, 13):
        current_start_date = datetime(start_year, month, 1)
        current_end_date = datetime(start_year, month + 1, 1) if month < 12 else datetime(start_year + 1, 1, 1)
        
        for parameter in parameters:
            has_more_data = True
            page = 1
            
            while has_more_data:
                try:
                    status, resp = api.measurements(city=city, parameter=parameter, limit=10000, page=page, date_from=current_start_date, date_to=current_end_date)
                    if status == 200 and resp['results']:
                        df = pd.DataFrame(resp['results'])
                        year_data_frames.append(df)
                        if len(df) < 10000:
                            has_more_data = False
                        else:
                            page += 1
                    else:
                        has_more_data = False
                except Exception as e:
                    print(f"Error encountered: {e}. Retrying in 60 seconds.")
                    time.sleep(60)
    
    year_data = pd.concat(year_data_frames, axis=0)
    return year_data


In [57]:
# initialize parameters we will be reusing for every year 
city = 'Delhi'
parameters = ['pm25', 'pm10', 'o3', 'co', 'so2', 'no2']

# get data for 2015
year_2015_data = fetch_data_for_year(city, parameters, 2015)
year_2015_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2015_data.csv', index=False)

In [59]:
# get data for 2016
year_2016_data = fetch_data_for_year(city, parameters, 2016)
year_2016_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2016_data.csv', index=False)

In [60]:
# get data for 2017
year_2017_data = fetch_data_for_year(city, parameters, 2017)
year_2017_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2017_data.csv', index=False)

In [62]:
year_2018_data = fetch_data_for_year(city, parameters, 2018)
year_2018_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2018_data.csv', index=False)

In [64]:
year_2019_data = fetch_data_for_year(city, parameters, 2019)
year_2019_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2019_data.csv', index=False)

In [65]:
year_2020_data = fetch_data_for_year(city, parameters, 2020)
year_2020_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2020_data.csv', index=False)

In [66]:
year_2021_data = fetch_data_for_year(city, parameters, 2021)
year_2021_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2021_data.csv', index=False)

In [68]:
year_2022_data = fetch_data_for_year(city, parameters, 2022)
year_2022_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2022_data.csv', index=False)

Error encountered: A bad request was made: 500. Retrying in 60 seconds.
Error encountered: A bad request was made: 500. Retrying in 60 seconds.


KeyboardInterrupt: 

In [71]:
# issues getting 2022 data as a full so will get by month and concatenate

def fetch_data_for_year2(city, parameters, start_year):
    api = openaq.OpenAQ()
    data_frames = []

    start_date = datetime(start_year, 1, 1)
    end_date = datetime(start_year + 1, 1, 1)

    for parameter in parameters:
        current_date = start_date
        while current_date < end_date:
            current_start_date = current_date
            current_end_date = min(current_date + timedelta(days=30), end_date)

            has_more_data = True
            page = 1

            while has_more_data:
                max_retries = 5
                retry_count = 0
                while retry_count < max_retries:
                    try:
                        status, resp = api.measurements(city=city, parameter=parameter, limit=10000, page=page, date_from=current_start_date, date_to=current_end_date)
                        if status == 200 and resp['results']:
                            df = pd.DataFrame(resp['results'])
                            data_frames.append(df)
                            if len(df) < 10000:
                                has_more_data = False
                            else:
                                page += 1
                        else:
                            has_more_data = False
                        break
                    except Exception as e:
                        retry_count += 1
                        print(f"Error encountered for {city}, {parameter}, {current_start_date}, {current_end_date}, Page: {page}. Error: {e}. Retrying in 60 seconds. Retry {retry_count} of {max_retries}.")
                        time.sleep(60)

                if retry_count == max_retries:
                    print(f"Max retries reached for {city}, {parameter}, {current_start_date}, {current_end_date}, Page: {page}. Skipping and moving to next.")
                    break

            current_date += timedelta(days=30)

    merged_data = pd.concat(data_frames, axis=0)
    return merged_data


city = 'Delhi'
parameters = ['pm25', 'pm10', 'o3', 'co', 'so2', 'no2']
start_year = 2022

year_2022_data = fetch_data_for_year2(city, parameters, start_year)


Error encountered for Delhi, co, 2022-01-01 00:00:00, 2022-01-31 00:00:00, Page: 1. Error: A bad request was made: 500. Retrying in 60 seconds. Retry 1 of 5.
Error encountered for Delhi, co, 2022-01-01 00:00:00, 2022-01-31 00:00:00, Page: 1. Error: A bad request was made: 500. Retrying in 60 seconds. Retry 2 of 5.
Error encountered for Delhi, co, 2022-01-01 00:00:00, 2022-01-31 00:00:00, Page: 1. Error: A bad request was made: 500. Retrying in 60 seconds. Retry 3 of 5.
Error encountered for Delhi, co, 2022-01-01 00:00:00, 2022-01-31 00:00:00, Page: 1. Error: A bad request was made: 500. Retrying in 60 seconds. Retry 4 of 5.
Error encountered for Delhi, co, 2022-01-01 00:00:00, 2022-01-31 00:00:00, Page: 1. Error: A bad request was made: 500. Retrying in 60 seconds. Retry 5 of 5.
Max retries reached for Delhi, co, 2022-01-01 00:00:00, 2022-01-31 00:00:00, Page: 1. Skipping and moving to next.


In [72]:
year_2022_data.to_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2022_data.csv', index=False)

In [79]:
# Load each CSV file
data_2015 = pd.read_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2015_data.csv')
data_2016 = pd.read_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2016_data.csv')
data_2017 = pd.read_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2017_data.csv')
data_2018 = pd.read_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2018_data.csv')
data_2019 = pd.read_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2019_data.csv')
data_2020 = pd.read_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2020_data.csv')
data_2021 = pd.read_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2021_data.csv')
data_2022 = pd.read_csv('/Users/magnesium/Documents/Light House Labs Bootcamp/Projects/Final Project/data/year_2022_data.csv')

# Concatenate all DataFrames into a single DataFrame
combined_data = pd.concat([data_2015, data_2016, data_2017, data_2018, data_2019, data_2020, data_2021, data_2022], axis=0)

# Save the combined DataFrame as a new CSV file
combined_data.to_csv('combined_data_2015_2022.csv', index=False)


In [80]:
combined_data.shape # HUGE DATAFRAME! 

(5432000, 8)